<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [0]:
# import statements
import pandas as pd
import numpy as np

In [0]:
# Let's upload the link and save it as a variable (df_data_url)
df_data_url='https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv'

In [3]:
# Now, using that variable create a dtaframe that pulls the data from the url, and print a sample
df=pd.read_csv(df_data_url)
df.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
356,357,20,RL,NaN,9248,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,1Story,6,6,1992,1992,Gable,CompShg,HdBoard,HdBoard,BrkFace,106.0,TA,TA,PConc,Gd,TA,No,GLQ,560,Unf,0,598,1158,GasA,...,Y,SBrkr,1167,0,0,1167,1,0,2,0,3,1,Gd,6,Typ,0,NaN,Attchd,1992.0,RFn,2,400,TA,TA,Y,120,26,0,0,0,0,NaN,NaN,NaN,0,7,2009,WD,Normal,173000
743,744,80,RL,70.0,12886,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,SLvl,5,6,1963,1999,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,Gd,TA,Av,ALQ,444,Unf,0,76,520,GasA,...,Y,SBrkr,1464,0,0,1464,0,1,2,0,3,1,TA,6,Min2,1,TA,Attchd,1997.0,RFn,2,480,TA,TA,Y,302,0,0,0,100,0,NaN,NaN,NaN,0,10,2009,WD,Normal,175000
1045,1046,20,RL,NaN,13680,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Edwards,Norm,Norm,1Fam,1Story,3,5,1955,1955,Hip,CompShg,BrkFace,Wd Sdng,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,...,Y,FuseA,1733,0,0,1733,0,0,2,0,4,1,TA,8,Min2,1,Gd,Attchd,1955.0,Unf,2,452,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal,139600
986,987,50,RM,59.0,5310,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Feedr,Norm,1Fam,1.5Fin,6,8,1910,2003,Hip,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,Fa,No,Unf,0,Unf,0,485,485,GasA,...,Y,SBrkr,1001,634,0,1635,0,0,1,0,2,1,Gd,5,Typ,0,NaN,Attchd,1950.0,Unf,1,255,Fa,TA,Y,394,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,117000
1410,1411,60,RL,79.0,12420,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,666,Unf,0,278,944,GasA,...,Y,SBrkr,944,896,0,1840,1,0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,RFn,2,622,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal,230000


In [4]:
# Now let's check for specific kinds of data
pd.set_option('display.max_rows',100) #this stroke commands to print more rows
df.dtypes #this commands to print the list of data types

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object


In [5]:
# To print a few rows of a particular column
df['LotConfig'].head(10)

0    Inside
1       FR2
2    Inside
3    Corner
4       FR2
5    Inside
6    Inside
7    Corner
8    Inside
9    Corner
Name: LotConfig, dtype: object

In [6]:
# Another way to call a column: dot method
df.LotConfig.head(10)

0    Inside
1       FR2
2    Inside
3    Corner
4       FR2
5    Inside
6    Inside
7    Corner
8    Inside
9    Corner
Name: LotConfig, dtype: object

In [7]:
# We can call a few different columns at once
df[['LotFrontage','YearBuilt']].head(5)

,LotFrontage,YearBuilt
0,65.0,2003
1,80.0,1976
2,68.0,2001
3,60.0,1915
4,84.0,2000


In [8]:
# But same logic won't work with dot-method
df.LotFrontage.YearBuilt.head(5)

AttributeError: ignored

In [9]:
# If most of the data in the column are integers but at least one of them is a string (like NaN)
#then the integers will turn into floats
df.LotFrontage.head(10)

0    65.0
1    80.0
2    68.0
3    60.0
4    84.0
5    85.0
6    75.0
7     NaN
8    51.0
9    50.0
Name: LotFrontage, dtype: float64

In [10]:
df['LotFrontage'].value_counts(dropna=False)

NaN      259
60.0     143
70.0      70
80.0      69
50.0      57
        ... 
106.0      1
38.0       1
138.0      1
140.0      1
137.0      1
Name: LotFrontage, Length: 111, dtype: int64

In [11]:
type(np.nan)

float

In [12]:
small_df=df[['TotalBsmtSF','1stFlrSF','2ndFlrSF','SalePrice']].copy()
small_df.head(5)
#.copy() insures that the new features are not connected to the originals

,TotalBsmtSF,1stFlrSF,2ndFlrSF,SalePrice
0,856,856,854,208500
1,1262,1262,0,181500
2,920,920,866,223500
3,756,961,756,140000
4,1145,1145,1053,250000


In [13]:
# Let's create a new column based n the data we already have
small_df['TotalSize']=small_df['TotalBsmtSF']+small_df['1stFlrSF']+small_df['2ndFlrSF']
small_df.head(5)

,TotalBsmtSF,1stFlrSF,2ndFlrSF,SalePrice,TotalSize
0,856,856,854,208500,2566
1,1262,1262,0,181500,2524
2,920,920,866,223500,2706
3,756,961,756,140000,2473
4,1145,1145,1053,250000,3343


In [14]:
# Other arythmetical operations work the same way
small_df['PricePerSq']=small_df['SalePrice']/small_df['TotalSize']
small_df.head(5)

,TotalBsmtSF,1stFlrSF,2ndFlrSF,SalePrice,TotalSize,PricePerSq
0,856,856,854,208500,2566,81.254871
1,1262,1262,0,181500,2524,71.909667
2,920,920,866,223500,2706,82.594235
3,756,961,756,140000,2473,56.611403
4,1145,1145,1053,250000,3343,74.783129


In [15]:
# NEW DATASET
! wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2020-01-21 21:18:36--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [        <=>         ]  21.72M   826KB/s    in 30s     

2020-01-21 21:19:06 (735 KB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22776791]



In [16]:
# Unzip the new file
! unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [31]:
# Create a new dataframe to work with this file
df_loan=pd.read_csv('LoanStats_2018Q4.csv', header=1, skipfooter=2, engine='python')
df_loan.shape

(128397, 144)

In [32]:
df_loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,6000,6000,6000.0,36 months,14.47%,206.44,C,C2,Manager,10+ years,RENT,55000.0,Verified,Dec-2018,Charged Off,n,NaN,NaN,credit_card,Credit card refinancing,104xx,NY,10.65,0,Jun-2006,0,NaN,NaN,6,0,4196,91.2%,8,w,0.00,0.00,1106.520000,1106.52,...,100.0,0,0,15800,12514,4300,11200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Oct-2019,ACTIVE,Oct-2019,3479.0,60.01,24.0
1,NaN,NaN,24000,24000,24000.0,36 months,10.33%,778.14,B,B1,Maintenance Manager,10+ years,MORTGAGE,100000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,606xx,IL,19.14,0,Oct-1996,3,39.0,NaN,14,0,8547,27.7%,28,w,16807.30,16807.30,9317.020000,9317.02,...,16.7,0,0,335121,50686,25900,61514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,30000,30000,30000.0,36 months,15.02%,1040.26,C,C3,Firefighter,10+ years,RENT,90000.0,Verified,Dec-2018,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,681xx,NE,22.23,0,May-2007,1,NaN,NaN,8,0,25754,48%,23,w,0.00,0.00,30883.673712,30883.67,...,50.0,0,0,89849,51370,41600,36249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,8000,8000,8000.0,36 months,12.98%,269.48,B,B5,Controller,4 years,MORTGAGE,95000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,480xx,MI,9.44,0,Mar-2009,2,73.0,NaN,7,0,11434,33.3%,20,w,1707.34,1707.34,6855.630000,6855.63,...,20.0,0,0,271550,25150,34300,14000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,22400,22400,22400.0,60 months,16.14%,546.40,C,C4,Paramedic/ Fire Fighter,10+ years,OWN,56000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,234xx,VA,18.47,0,Dec-2004,0,75.0,NaN,15,0,13555,38.2%,22,w,19230.91,19230.91,6526.670000,6526.67,...,25.0,0,0,253539,32070,28200,23364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Check for missing data
pd.set_option('display.max_rows', 200)
df_loan.isnull().sum().sort_values(ascending=False)

id                                            128397
desc                                          128397
member_id                                     128397
url                                           128397
orig_projected_additional_accrued_interest    128191
hardship_type                                 128164
hardship_last_payment_amount                  128164
hardship_payoff_balance_amount                128164
hardship_reason                               128164
hardship_loan_status                          128164
hardship_dpd                                  128164
hardship_length                               128164
payment_plan_start_date                       128164
hardship_end_date                             128164
hardship_start_date                           128164
hardship_amount                               128164
deferral_term                                 128164
hardship_status                               128164
settlement_percentage                         

In [0]:
# Dropping colums
dfl=df_loan.drop(['url','member_id','desc','id'], axis=1).copy()

In [35]:
dfl['int_rate'].head(10)

0     14.47%
1     10.33%
2     15.02%
3     12.98%
4     16.14%
5     12.98%
6     15.02%
7     16.14%
8     14.47%
9     13.56%
Name: int_rate, dtype: object

In [36]:
dfl['int_rate'][3]

' 12.98%'

In [37]:
type(dfl['int_rate'][3])

str

In [0]:
int_rate=' 12.98%'

In [40]:
int_rate.strip()

'12.98%'

In [41]:
int_rate.strip('%')

' 12.98'

In [42]:
int_rate.strip().strip('%')

'12.98'

In [43]:
float('12.98')

12.98

In [44]:
float(int_rate.strip().strip('%'))

12.98

In [45]:
type(float(int_rate.strip().strip('%')))

float

In [0]:
# Let's do all that at once by using a function
def myfunc(cell_contents):
  return float(cell_contents.strip().strip('%'))

In [52]:
# Let's now call the function
myfunc(int_rate)

12.98

# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01